# Step 1
Import modules

In [1]:
''' import modules '''
import time
import argparse
import os.path as osp

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


from torchvision import datasets, transforms
from logger import Logger



# Step 2
Define an easy network

In [2]:
# Fully connected neural network with three hidden layer
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 320)
        self.fc2 = nn.Linear(320, 160)
        self.fc3 = nn.Linear(160, 80)
        self.fc4 = nn.Linear(80, 40)
        self.fc5 = nn.Linear(40, 10)
        

    def forward(self, x):
        x = x.view(-1,28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x)
    

model = Net() 
model.cuda()

Net(
  (fc1): Linear(in_features=784, out_features=320, bias=True)
  (fc2): Linear(in_features=320, out_features=160, bias=True)
  (fc3): Linear(in_features=160, out_features=80, bias=True)
  (fc4): Linear(in_features=80, out_features=40, bias=True)
  (fc5): Linear(in_features=40, out_features=10, bias=True)
)

# Step 3
Hyper-parameters and data

In [3]:
''' Hyper-parameters'''
epochs = 10
batch_size = 64
lr = 0.01
momentum = 0.5
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [4]:
''' Prapare data'''
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))]
                   )),
    batch_size=batch_size, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))]
                   )),
    batch_size=batch_size, shuffle=True, **kwargs)


# Step 4
Define train and validation function

In [5]:
''' Train and test'''

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # print(data.shape) [64,1,28,28]
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()

    # loss function already averages over batch size
    test_loss /= len(test_loader)
    acccuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, '
          'Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss,
                                               correct,
                                               len(test_loader.dataset),
                                               acccuracy))
    return test_loss, acccuracy, data

def test_train(epoch):
    model.eval()
    train_loss = 0
    correct = 0
    for data, target in train_loader:
        
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()

    # loss function already averages over batch size
    train_loss /= len(train_loader)
    acccuracy = 100. * correct / len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, '
          'Accuracy: {}/{} ({:.0f}%)\n'.format(train_loss,
                                               correct,
                                               len(train_loader.dataset),
                                               acccuracy))
    return train_loss, acccuracy, data


# Step 5
Train and record

In [ ]:
''' train'''
best_loss = None
logger_train = Logger('./logs/Momentum/0.5_trian')
logger_val = Logger('./logs/Momentum/0.5_val')
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(epoch)
    test_loss, test_acc, images = test(epoch)
    train_loss, train_acc, _ = test_train(epoch)
    print('-' * 89)
    timeLength = time.time() - epoch_start_time
    print('| end of epoch {:3d} | time: {:5.2f}s '.format(
        epoch, time.time() - epoch_start_time))
    print('-' * 89)
    best_loss = test_loss
    with open('model.pt', 'wb') as fp:
            state = model.state_dict()
            torch.save(state, fp)
    # ================================================================== #
    #                        Tensorboard Logging                         #
    # ================================================================== #
    # 1. Log scalar values (scalar summary)
    info = { 'loss': test_loss, 'accuracy': test_acc }
    for tag, value in info.items():
        logger_val.scalar_summary(tag, value, epoch+1)
    info = { 'loss': train_loss, 'accuracy': train_acc }
    for tag, value in info.items():
        logger_train.scalar_summary(tag, value, epoch+1)

    # 2. Log values and gradients of the parameters (histogram summary)
    for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            logger_train.histo_summary(tag, value.data.cpu().numpy(), epoch+1)
            logger_train.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), epoch+1)

    # 3. Log training images (image summary)
    info = { 'images': images.view(-1, 28, 28)[:10].cpu().numpy() }

    for tag, images in info.items():
            logger_val.image_summary(tag, images, epoch+1)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.335186
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.276842
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.211098
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.136191
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.363432
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.844934
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.537819
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.529223
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.449302
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.498455


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:27: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:29: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.3786, Accuracy: 8868/10000 (88%)



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:51: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:53: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Train set: Average loss: 0.3850, Accuracy: 53152/60000 (88%)

-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 14.01s 
-----------------------------------------------------------------------------------------
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.413723
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.192235
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.365040
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.222258
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.400789
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.367563
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.256143
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.358769
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.268255


In [ ]:
done = True